In [7]:
import os
import pandas as pd

# Get the current directory
current_directory = os.getcwd()

# Define the path for the Excel file
excel_file_path = r'C:\Users\lesli\Documents\Duke\Masters\AIPI549\FinalProcessed-LinkedIn-2-23-2024-4101963-all-leads (002).xlsx'

# Read the Excel file, specifying data types to prevent conversion
df = pd.read_excel(excel_file_path, dtype={'Company Size': str})

# Define the path for the CSV file inside the current directory
csv_file_name = 'FinalProcessed-LinkedIn-2-23-2024-4101963-all-leads.csv'
csv_file_path = os.path.join(current_directory, csv_file_name)

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print("CSV file has been created successfully at:", csv_file_path)

CSV file has been created successfully at: C:\Users\lesli\Documents\Duke\Masters\AIPI549\DCP_Capstone\NeuralNetworkClassifier\FinalProcessed-LinkedIn-2-23-2024-4101963-all-leads.csv


In [88]:
import datetime
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

csv_file_name = 'FinalProcessed-LinkedIn-2-23-2024-4101963-all-leads.csv'
csv_file_path = os.path.join(current_directory, csv_file_name)

df = pd.read_csv(csv_file_path, dtype={'Company Size': str})

hot_encode_columns = [
    'Lead Job Title',
    'Company Size',
    'Company Industry',
    'Company Li Company Type',
    'Company Location Country Name'
]

df_encoded = pd.DataFrame()

current_year = datetime.datetime.now().year
df_encoded['Years Since Company Founded'] = current_year - df['Company Founded In']

# Convert Lead Years and Months to total months for position
df_encoded['Total Months In Position'] = df['Lead Years In Position'] * 12 + df['Lead Months In Position']

# Convert Lead Years and Months to total months for company tenure
df_encoded['Total Months In Company'] = df['Lead Years In Company'] * 12 + df['Lead Months In Company']

for column in hot_encode_columns+['relevant', 'Company Followers']:
    df_encoded[column] = df[column]

df_encoded = pd.get_dummies(df_encoded, columns=hot_encode_columns)

df_encoded.fillna(0, inplace=True)

df_encoded['relevant'] = df['relevant'].replace(2, 1)

In [89]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert boolean columns to integers (0 or 1)
boolean_columns = df_encoded.select_dtypes(include='bool').columns
df_encoded[boolean_columns] = df_encoded[boolean_columns].astype(int)

# Convert DataFrame to PyTorch tensors
X = df_encoded.drop(columns=['relevant']).values
y = df_encoded['relevant'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Define neural network architecture, hyperparameters, training, and evaluation as before...


class LargerNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(LargerNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Define hyperparameters
input_size = X_train.shape[1]
hidden_size1 = 128  # Increased number of neurons in the first hidden layer
hidden_size2 = 64   # You can adjust this as well
num_classes = 3
learning_rate = 0.001
num_epochs = 100

# Initialize the model, loss function, and optimizer
model = LargerNeuralNetwork(input_size, hidden_size1, hidden_size2, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f'Accuracy on test set: {accuracy:.4f}')

Epoch [1/100], Loss: 53.7275
Epoch [2/100], Loss: 21.9468
Epoch [3/100], Loss: 24.6178
Epoch [4/100], Loss: 24.9930
Epoch [5/100], Loss: 24.2304
Epoch [6/100], Loss: 22.7477
Epoch [7/100], Loss: 20.5903
Epoch [8/100], Loss: 17.9184
Epoch [9/100], Loss: 14.3649
Epoch [10/100], Loss: 10.8742
Epoch [11/100], Loss: 7.0475
Epoch [12/100], Loss: 2.9286
Epoch [13/100], Loss: 97.4067
Epoch [14/100], Loss: 4.7250
Epoch [15/100], Loss: 10.1652
Epoch [16/100], Loss: 14.8360
Epoch [17/100], Loss: 19.0347
Epoch [18/100], Loss: 22.5627
Epoch [19/100], Loss: 25.5361
Epoch [20/100], Loss: 27.9344
Epoch [21/100], Loss: 29.9622
Epoch [22/100], Loss: 31.7467
Epoch [23/100], Loss: 33.2965
Epoch [24/100], Loss: 34.0334
Epoch [25/100], Loss: 34.7484
Epoch [26/100], Loss: 35.1902
Epoch [27/100], Loss: 35.3790
Epoch [28/100], Loss: 35.3066
Epoch [29/100], Loss: 34.9655
Epoch [30/100], Loss: 34.4448
Epoch [31/100], Loss: 33.7195
Epoch [32/100], Loss: 32.7780
Epoch [33/100], Loss: 31.6477
Epoch [34/100], Loss: 

In [90]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    print(f'Accuracy on test set: {accuracy:.4f}')
    
    # Confusion matrix
    cm = confusion_matrix(y_test_tensor.numpy(), predicted.numpy())
    print("Confusion Matrix:")
    print(cm)
    
    # Classification report
    report = classification_report(y_test_tensor.numpy(), predicted.numpy())
    print("\nClassification Report:")
    print(report)

Accuracy on test set: 0.7063
Confusion Matrix:
[[96 15]
 [27  5]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       111
           1       0.25      0.16      0.19        32

    accuracy                           0.71       143
   macro avg       0.52      0.51      0.51       143
weighted avg       0.66      0.71      0.68       143

